In [45]:
START_STATE_KEY = "START"
STOP_STATE_KEY = "STOP"

# Given filename, return list of sentences, each comprising list of [word,token] for training and list of word for testing
def parse_dataset(file,is_training=True):
    with open(file, "r", encoding="utf8") as f:
        data = f.read()
    sentences = data.split('\n\n')
    sentences = [i.strip().split('\n') for i in sentences if not i=='']
    if is_training:
        sentences = [[j.strip().split(' ') for j in i] for i in sentences]
        sentences = [[["".join(j[:-1]),j[-1]] for j in i] for i in sentences]
    return sentences

# Get list of words and list of states
def enumerate_words_and_states(training_sentences):
    words = set()
    states = set()
    for i in training_sentences:
        for j in i:
            words.add(j[0])
            states.add(j[1])
    states.add(START_STATE_KEY)
    states.add(STOP_STATE_KEY)
    return list(words),list(states)

def get_word_frequency(training_sentences):
    out_dict = dict()
    for i in training_sentences:am in my room now
        for j in i:
            if not j[0] in out_dict:
                out_dict[j[0]] = 0
            out_dict[j[0]] += 1
    return out_dict

def get_log_emission_params(words,states,training_sentences,word_frequency):
    count_dict = {i:{j:0 for j in words} for i in states if not (i==START_STATE_KEY or i==STOP_STATE_KEY)}
    for i in sentences:
        for j in i:
            count_dict[j[1]][j[0]] += 1
    for i in count_dict:
        total_count = sum([j[1] for j in count_dict[i].items()])

In [46]:
sentences = parse_dataset("data/ES/train")
enumerate_words_and_states(sentences)

(['camarera',
  'EVITADLO',
  'tanta',
  'pedirlo',
  'triunfado',
  'Groupalia',
  'cobrarnos',
  'ganar',
  'atencion',
  'suegros',
  'diferencian',
  'celebramos',
  'pides',
  'consumo',
  'afectuoso',
  'vigilando',
  'nuetro',
  'bochornosa',
  'entrantres',
  'íré',
  'padre',
  'coincido',
  'solucitud',
  'Mesa',
  'llevarse',
  'repito',
  'sentó',
  'slago',
  'euros.El',
  'anterioridad',
  'quiero',
  'a',
  'ahí',
  'entusiasmó',
  'detalle',
  'Muelle',
  'escalones',
  'extraordinario',
  'extraordinarios',
  'ZAAGOZA',
  'sobraba',
  'junio',
  'recibio',
  'hablo',
  'hamburguesas',
  'salvo',
  'ExcelentesDe',
  'grafitis',
  'llevaron',
  'retsaurante',
  'quedamos',
  'cara',
  'Buenisimo',
  'apoyo',
  'acertada',
  'ginebra',
  'eficiente',
  'confirmacion',
  'Destaca',
  'idiomas',
  'Primera',
  'sabrosísimo',
  'comí',
  'michelin',
  'corral',
  'pusieran',
  'generosidad',
  'respetos',
  'mantienen',
  'Producto',
  'encebollado.Risotto',
  'magnífica',
 

In [10]:
with open("data/ES/train","rb") as f:
    data = f.read()


In [29]:
data[4100:4120]

b' O\r\n, O\r\n\xe2\x80\x9c O\r\n\xc3\xbani'

In [47]:
ls = [("a","b"),("b","c"),("a","b"),("a","b"),("a","b")]
ls.count(("a","b"))

4